## Example: Initialization and Primitives of the Executor

This example shows different ways to initialize the Executor und run various jobs.

In [1]:
from qiskit import Aer
from qiskit.circuit.random import random_circuit
from qiskit.primitives import Sampler, Estimator, BackendSampler, BackendEstimator
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_ibm_runtime import Estimator as RuntimeEstimator

from squlearn.util import Executor

The following cell shows different ways to initialize of the ``Executor`` class from different backends or services.

In [2]:
# from a string representing the simple Qiskit Aer simulators:
executor = Executor("statevector_simulator")
executor = Executor("qasm_simulator")

# from a backend following the Qiskit backend standard:
executor = Executor(Aer.get_backend("statevector_simulator"))

self._shots None
self._shots None
self._shots None


In [3]:
# from a backend obtained from the Qiskit IBM runtime service (here, the account has to be set-up previously):
service = QiskitRuntimeService(token = "1e1fcef3940bc7181262dbc135a052e83b17a669f187b1d6d7bde96ef3167078b55eb494f44f3b3de8ea934023da7a5eef3df4c6d27bf8ce7d48ca174c654cb7")#channel="ibm_cloud")
# Alternative: service = QiskitRuntimeService(channel="ibm_quantum",token="YOUR_TOKEN_HERE")
executor = Executor(service.get_backend("ibm_nairobi"))

# from a session set-up with the Qiskit IBM runtime service:
session = Session(service, backend=service.get_backend("ibm_nairobi"), max_time=28800)
executor = Executor(session)

# from the Qiskit IBM runtime Estimator primitive:
session = Session(service, backend=service.get_backend("ibm_nairobi"), max_time=28800)
print(session)
estimator = RuntimeEstimator(session)
executor = Executor(estimator)

ConnectionError: HTTPSConnectionPool(host='iam.auth.quantum-computing.ibm.com', port=443): Max retries exceeded with url: /identity/token (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001A6173FA500>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
# from a Qiskit simulator primitive:
executor = Executor(Estimator())
executor = Executor(Sampler())
executor = Executor(BackendEstimator(Aer.get_backend("qasm_simulator")))
executor = Executor(BackendSampler(Aer.get_backend("qasm_simulator")))

The cells demonstrates how to set the number of shots utilized in the circuit evaluation.

In [ ]:
# Shots can be set by the executor:
print("Current shots as set before:", executor.get_shots())
# Set shots
executor.set_shots(1234)
print("Adjusted shots:", executor.get_shots())
# Reset shots to initial ones:
executor.reset_shots()
print("Reset shots:", executor.get_shots())

In this cell, we calculate an expectation value using the Estimator primitive, which is accessible through the ``Executor`` class. The executor generates modified Primitives with enhanced functionality, including caching, automatic session management, and logging capabilities. These modified primitives can be seamlessly incorporated into your workflow or other Qiskit routines.

In [ ]:
# Generate a random circuit:
circuit = random_circuit(2, 2, seed=0).decompose(reps=1)

# Generate an observable:
observable = SparsePauliOp("ZI")

# Get the Executor Estimator Primitive and call run:
estimator = executor.get_estimator()
print(estimator.run(circuit, observable, shots=4321).result())

# Get the Executor Sampler Primitive and call run:
sampler = executor.get_sampler()
print(sampler.run(circuit.measure_all(inplace=False)).result())

The executor can also be used to execute ``backend.run()``. However, caching is not yet implemented for this case.

In [ ]:
job = executor.backend_run(circuit)
job.result()

Additionally, the executor maintains a detailed log of background operations. This feature proves especially valuable when optimizing real backends, allowing you to gain insights into the underlying processes and activities.

In [ ]:
executor = Executor("qasm_simulator", log_file="example_log.log")
executor.set_shots(1234)
estimator = executor.get_estimator()

print(estimator.run(circuit, observable, shots=4321).result())

The executor has a cache where it stores and can reuse job results. In this example, we change the number of shots to tell apart the first and second runs of the same job, and both runs are stored in separate caches. However, the third job simply reuses the cached result from its first execution.

In [ ]:
executor = Executor(
    BackendSampler(Aer.get_backend("qasm_simulator")),
    log_file="example_log_cache.log",
    caching=True,
    cache_dir="_cache",
)
executor.set_shots(4321)
estimator = executor.get_estimator()
print(estimator.run(circuit, observable).result())
executor.set_shots(1234)
print(estimator.run(circuit, observable).result())
# This one is load from the cached and not executed again
executor.set_shots(4321)
print(estimator.run(circuit, observable).result())